In [86]:
import math
import random
import csv
import pandas as pd

In [87]:
# the categorical class names are changed to numberic data
def encode_class(mydata):
    classes = []
    for i in range(len(mydata)):
        if mydata[i][-1] not in classes:
            classes.append(mydata[i][-1])
    for i in range(len(classes)):
        for j in range(len(mydata)):
            if mydata[j][-1] == classes[i]:
                mydata[j][-1] = i
    return mydata 

In [88]:
# Splitting the data
def splitting(mydata, ratio):
    train_num = int(len(mydata) * ratio)
    train = []
    # initially testset will have all the dataset
    test = list(mydata)
    while len(train) < train_num:
        # index generated randomly from range 0
        # to length of testset
        index = random.randrange(len(test))
        # from testset, pop data rows and put it in train
        train.append(test.pop(index))
    return train, test

In [89]:
# Group the data rows under each class yes or  no in dictionary
def groupUnderClass(mydata):
      dict = {}
      for i in range(len(mydata)):
          if (mydata[i][-1] not in dict):
              dict[mydata[i][-1]] = []
          dict[mydata[i][-1]].append(mydata[i])
      return dict
 

In [90]:
# Calculating Mean
def mean(numbers):
    return sum(numbers) / float(len(numbers))
 

In [91]:
# Calculating Standard Deviation
def std_dev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    
    return math.sqrt(variance)

In [92]:
def MeanAndStdDev(mydata):
    info = [(mean(attribute), std_dev(attribute)) for attribute in zip(*mydata)]
    del info[-1]
    return info

In [93]:
def MeanAndStdDevForClass(mydata):
    info = {}
    dict = groupUnderClass(mydata)
    for classValue, instances in dict.items():
        info[classValue] = MeanAndStdDev(instances)
    return info

In [94]:
# Calculate Gaussian Probability Density Function
def calculateGaussianProbability(x, mean, stdev):
    expo = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * expo

In [95]:
# Calculate Class Probabilities
def calculateClassProbabilities(info, test):
    probabilities = {}
    for classValue, classSummaries in info.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, std_dev = classSummaries[i]
            x = test[i]
            probabilities[classValue] *= calculateGaussianProbability(x, mean, std_dev)
    return probabilities

In [96]:
# Make prediction - highest probability is the prediction
def predict(info, test):
    probabilities = calculateClassProbabilities(info, test)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [97]:
# returns predictions for a set of examples
def getPredictions(info, test):
    predictions = []
    for i in range(len(test)):
        result = predict(info, test[i])
        predictions.append(result)
    return predictions

In [98]:
# Accuracy score
def accuracy_rate(test, predictions):
    correct = 0
    for i in range(len(test)):
        if test[i][-1] == predictions[i]:
            correct += 1
    return (correct / float(len(test))) * 100.0

In [99]:
df = pd.read_csv('CM1.arff')
df

,6,9,2,1,0,16,5,0.2,8,2.1,...,14,0.16,19,44,15,15.1,32,4.1,25,0.2.2
0,15,7,3,1,19,12,4,0.13,6,2.00,...,15,0.06,51,90,32,27,67,39.22,32,1
1,27,9,1,4,22,16,5,0.15,8,2.00,...,15,0.06,37,74,33,22,83,47.27,33,1
2,7,3,2,0,0,4,2,0.17,2,2.00,...,6,0.10,9,14,7,10,20,0.00,12,0
3,51,25,13,0,14,48,13,0.12,24,2.00,...,85,0.08,192,229,71,20,172,11.67,106,0
4,3,5,2,0,0,8,3,0.20,4,2.00,...,11,0.16,15,21,5,12,19,0.00,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,17,21,4,1,8,36,11,0.26,16,2.25,...,34,0.16,98,143,45,33,68,18.00,42,0
339,16,13,6,0,5,24,7,0.12,12,2.00,...,26,0.09,186,263,77,26,81,7.81,59,0
340,11,9,6,0,4,16,5,0.12,8,2.00,...,21,0.09,80,129,41,19,58,8.70,42,0
341,1,3,0,0,0,4,2,0.22,2,2.00,...,5,0.18,4,11,4,7,11,0.00,9,0


In [100]:
df.shape

(343, 38)

In [102]:
# add the data path in your system
filename = 'CM1.arff'
 
 
# load the file and store it in mydata list
mydata = csv.reader(open(filename))
mydata = list(mydata)
mydata = encode_class(mydata)
for i in range(len(mydata)):
    mydata[i] = [float(x) for x in mydata[i]]
 
     
# split ratio = 0.7
# 70% of data is training data and 30% is test data used for testing
ratio = 0.7
train_data, test_data = splitting(mydata, ratio)
print('Total number of examples are: ', len(mydata))
print('Out of these, training examples are: ', len(train_data))
print("Test examples are: ", len(test_data))
 
# prepare model
info = MeanAndStdDevForClass(train_data)
 
# test model
predictions = getPredictions(info, test_data)
accuracy = accuracy_rate(test_data, predictions)
print("Accuracy of your model is: ", accuracy)

Total number of examples are:  344
Out of these, training examples are:  240
Test examples are:  104
Accuracy of your model is:  83.65384615384616
